In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('webscraping-github')
jovian.set_colab_id('1YEE_QTwm0jgL-yalvU_c916p-v-W0ZDp')

     |████████████████████████████████| 68 kB 3.0 MB/s 


# webscraping-github

Use the "Run" button to execute the code.

In [2]:
!pip install jovian --upgrade --quiet

In [3]:
import jovian

In [4]:
# Execute this to save new versions of the notebook
jovian.commit(project="webscraping-github")

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/vedashree-gadekar/webscraping-github


'https://jovian.ai/vedashree-gadekar/webscraping-github'

### Picking a website and general overview of the project
- Scraping the following website: 'https://github.com/topics'
- Getting list of the topics. For every topic, getting topic title, topic page URL and description.
- For every topic, getting top 25 repositories in the topic from the topic page.
- For each repository, grabbing the repository name, username, stars and repository URL.
- For each topic, creating the CSV file of the following format:
``````
Repo Name,Username,Stars,Repo URL
infinite-scroll,metafizzy,7100,https://github.com/metafizzy/infinite-scroll
Blog,ljianshu,7000,https://github.com/ljianshu/Blog
``````

In [5]:
# importing requests library
import requests

In [6]:
# webpage selected for scraping
topics_url = 'https://github.com/topics'

In [7]:
response = requests.get(topics_url)

In [8]:
response.status_code

200

In [9]:
len(response.text)

139421

In [10]:
page_contents = response.text

In [11]:
page_contents[:1000]

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" >\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-ksfTgQOOnE+FFXf+yNfVjKSlEckJAdufFIYGK7ZjRhWcZgzAGcmZqqArTgMLpu90FwthqcCX4ldDgKXbmVMeuQ==" rel="stylesheet" href="https://github.githubassets.com/assets/light-92c7d381038e.css" /><link crossorigin="anonymous" media="all" integrity="sha512-1KkMNn8M/al/dtzBLupRwkIOgnA9MWkm8oxS+solP87jByEvY/g4BmoxLihRogKcX1obPnf4Yp7dI0ZTWO+ljg==" rel="styl

In [12]:
# creating a local copy of the webpage on the server
with open('webpage.html', 'w') as f:
    f.write(page_contents)

### Using Beautiful Soup for parsing and extracting information

In [13]:
!pip install beautifulsoup4 --upgrade --quiet

     |████████████████████████████████| 128 kB 5.2 MB/s 


In [14]:
from bs4 import BeautifulSoup

In [15]:
# parsing information using html.parser
doc = BeautifulSoup(page_contents, 'html.parser')

In [16]:
# getting the topic title of the topics from the website by inspecting it
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary' 

topic_title_tags = doc.find_all('p', {'class': selection_class})

In [17]:
len(topic_title_tags)

30

In [18]:
topic_title_tags[:5]

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>]

In [19]:
# getting the topic description of the topics from the website by inspecting it
desc_selector = 'f5 color-fg-muted mb-0 mt-1'

topic_desc_tags = doc.find_all('p', {'class': desc_selector})

In [20]:
len(topic_desc_tags)

30

In [21]:
topic_desc_tags[:5]

[<p class="f5 color-fg-muted mb-0 mt-1">
           3D modeling is the process of virtually developing the surface and structure of a 3D object.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Ajax is a technique for creating interactive web applications.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Algorithms are self-contained sequences that carry out a variety of tasks.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Amp is a non-blocking concurrency library for PHP.
         </p>, <p class="f5 color-fg-muted mb-0 mt-1">
           Android is an operating system built by Google designed for mobile devices.
         </p>]

In [22]:
# getting the topic title tag at the 0th element which is 3D
topic_title_tag0 = topic_title_tags[0]

In [23]:
topic_title_tag0

<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>

In [24]:
# getting the parent tag of the above mentioned tag ie looking up inside which tag does the above tag lie
div_tag = topic_title_tag0.parent

In [25]:
div_tag

<a class="no-underline flex-1 d-flex flex-column" href="/topics/3d">
<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>
<p class="f5 color-fg-muted mb-0 mt-1">
          3D modeling is the process of virtually developing the surface and structure of a 3D object.
        </p>
</a>

In [26]:
# getting the link URL of the topics from the a tag (parent of the p tags for topic titles and descriptions)
topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column' })

In [27]:
len(topic_link_tags)

30

In [28]:
# printing URL of the first topic (3D) on the GitHub page
topic0_url = 'https://github.com' + topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [29]:
# printing only the title of the topics
topic_titles = [] #created an empty dictionary for title tags
for tag in topic_title_tags:
    topic_titles.append(tag.text)
print(topic_titles)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [30]:
# parsing description of the topics
topic_descs = []
for tag in topic_desc_tags:
    topic_descs.append(tag.text.strip()) # strip removes the unnecessary space present in the text
topic_descs

['3D modeling is the process of virtually developing the surface and structure of a 3D object.',
 'Ajax is a technique for creating interactive web applications.',
 'Algorithms are self-contained sequences that carry out a variety of tasks.',
 'Amp is a non-blocking concurrency library for PHP.',
 'Android is an operating system built by Google designed for mobile devices.',
 'Angular is an open source web application platform.',
 'Ansible is a simple and powerful automation engine.',
 'An API (Application Programming Interface) is a collection of protocols and subroutines for building software.',
 'Arduino is an open source hardware and software company and maker community.',
 'ASP.NET is a web framework for building modern web apps and services.',
 'Atom is a open source text editor built with web technologies.',
 'An awesome list is a list of awesome things curated by the community.',
 'Amazon Web Services provides on-demand cloud computing platforms on a subscription basis.',
 'Azu

In [31]:
# parsing topic URLs
topic_urls = []
base_url = 'https://github.com'
for tag in topic_link_tags:
    topic_urls.append(base_url + tag['href'])
topic_urls

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/atom',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compil

### Creating CSV file/s with the extracted information

In [32]:
!pip install pandas --quiet

In [33]:
import pandas as pd

In [34]:
# creating topics dictionary to display the data in a table using pandas library
topics_dict = {'Topic': topic_titles, # 'Topic'-name of the column, topic_titles-info of that attribute to be stored in the column 
               'Description': topic_descs,
               'URL': topic_urls }

In [35]:
topics_df = pd.DataFrame(topics_dict)

In [36]:
topics_df

,Topic,Description,URL
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android
5,Angular,Angular is an open source web application plat...,https://github.com/topics/angular
6,Ansible,Ansible is a simple and powerful automation en...,https://github.com/topics/ansible
7,API,An API (Application Programming Interface) is ...,https://github.com/topics/api
8,Arduino,Arduino is an open source hardware and softwar...,https://github.com/topics/arduino
9,ASP.NET,ASP.NET is a web framework for building modern...,https://github.com/topics/aspnet


In [37]:
# creating CSV file of topics
topics_df.to_csv('topics.csv', index = None) # setting index=None so that the index no doesnt appear at start of each element

### Getting information out of a topic page


In [38]:
# getting the url of each topic page (3D page here)
topic_page_url = topic_urls[0]

In [39]:
topic_page_url

'https://github.com/topics/3d'

In [40]:
# geting response of topic page
response = requests.get(topic_page_url)

In [41]:
response.status_code

200

In [42]:
# checking the no of characters on the 3D topic page
len(response.text)

631316

In [43]:
doc = BeautifulSoup(response.text, 'html.parser')

In [44]:
# parsing the username of the top 30 repositories from the 3D topic page which lies inside h3 tag
h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed' 

repo_tags = doc.find_all('h3', {'class': h3_selection_class})

In [45]:
repo_tags

[<h3 class="f3 color-fg-muted text-normal lh-condensed">
 <a data-ga-click="Explore, go to repository owner, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
   
             mrdoob
 
   
 </a>          /
           <a class="text-bold wb-break-word" data-ga-click="Explore, go to repository, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-h

In [46]:
len(repo_tags)

30

In [47]:
repo_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a data-ga-click="Explore, go to repository owner, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
  
            mrdoob

  
</a>          /
          <a class="text-bold wb-break-word" data-ga-click="Explore, go to repository, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-cli

In [48]:
# finding all 'a' tags in the first repo tag
a_tags = repo_tags[0].find_all('a')

In [49]:
# parsing username of the first repository from the 3D page using the 'a' tag
a_tags[0].text.strip()

'mrdoob'

In [50]:
# parsing the repo name of the first repo using the 'a' tag
a_tags[1].text.strip()

'three.js'

In [51]:
base_url = 'https://github.com'
repo_url = base_url + a_tags[1]['href']

In [52]:
# printing the URL of the first repo from the 3D topic webpage
print(repo_url)

https://github.com/mrdoob/three.js


In [53]:
# getting the no of stars from the repsoitory
star_tags = doc.find_all('span', {'class': 'Counter js-social-count'})

In [54]:
len(star_tags)

30

In [55]:
star_tags[0].text.strip()

'80.9k'

In [56]:
# function to convert stars string to integer
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1]) * 1000)
    return int(stars_str)

In [57]:
parse_star_count(star_tags[0].text.strip())

80900

In [58]:
# function to return info about a specific repository
def get_repo_info(h3_tag, star_tag):
    # returns all the required info about a repository
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, repo_url, stars

In [59]:
get_repo_info(repo_tags[0], star_tags[0])

('mrdoob', 'three.js', 'https://github.com/mrdoob/three.js', 80900)

In [60]:
# function to find repo info about all the repos (top 30 in this case)
topic_repos_dict = {
    'username': [],
    'repo_name': [],
    'repo_url': [],
    'stars': []
}
for i in range(len(repo_tags)):
    repo_info = get_repo_info(repo_tags[i], star_tags[i])
    topic_repos_dict['username'].append(repo_info[0])
    topic_repos_dict['repo_name'].append(repo_info[1])
    topic_repos_dict['stars'].append(repo_info[2])
    topic_repos_dict['repo_url'].append(repo_info[3])
    

In [61]:
import os
# function to return topic page
def get_topic_page(topic_url):
    # Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page{}'.format(topic_url))
    # Parse using Beautiful Soup 
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc

# function to return info about a specific repository
def get_repo_info(h3_tag, star_tag):
    # returns all the required info about a repository
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

def get_topic_repos(topic_doc):
    # Get the h3 tag containing repo title, repo URL and username
    h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed' 
    repo_tags = topic_doc.find_all('h3', {'class': h3_selection_class})
    # Get star tags
    star_tags = topic_doc.find_all('span', {'class': 'Counter js-social-count'})
    
    topic_repos_dict = {
        'username': [],
        'repo_name': [],
        'repo_url': [],
        'stars': []
    }
    
    # Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i], star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['repo_url'].append(repo_info[2])
        topic_repos_dict['stars'].append(repo_info[3])
    
    return pd.DataFrame(topic_repos_dict)

In [62]:
# checking if the function is working
url4 = topic_urls[4]

In [63]:
url4

'https://github.com/topics/android'

In [64]:
topic4_doc = get_topic_page(url4)

In [65]:
topic4_repos = get_topic_repos(topic4_doc)

In [66]:
topic4_repos 

,username,repo_name,repo_url,stars
0,flutter,flutter,139000,https://github.com/flutter/flutter
1,justjavac,free-programming-books-zh_CN,90600,https://github.com/justjavac/free-programming-...
2,Genymobile,scrcpy,63700,https://github.com/Genymobile/scrcpy
3,Hack-with-Github,Awesome-Hacking,50400,https://github.com/Hack-with-Github/Awesome-Ha...
4,google,material-design-icons,45500,https://github.com/google/material-design-icons
5,wasabeef,awesome-android-ui,42400,https://github.com/wasabeef/awesome-android-ui
6,square,okhttp,41900,https://github.com/square/okhttp
7,android,architecture-samples,40500,https://github.com/android/architecture-samples
8,Solido,awesome-flutter,40100,https://github.com/Solido/awesome-flutter
9,square,retrofit,39800,https://github.com/square/retrofit


In [67]:
# writing the function in a single line
get_topic_repos(get_topic_page(topic_urls[4]))

,username,repo_name,repo_url,stars
0,flutter,flutter,139000,https://github.com/flutter/flutter
1,justjavac,free-programming-books-zh_CN,90600,https://github.com/justjavac/free-programming-...
2,Genymobile,scrcpy,63700,https://github.com/Genymobile/scrcpy
3,Hack-with-Github,Awesome-Hacking,50400,https://github.com/Hack-with-Github/Awesome-Ha...
4,google,material-design-icons,45500,https://github.com/google/material-design-icons
5,wasabeef,awesome-android-ui,42400,https://github.com/wasabeef/awesome-android-ui
6,square,okhttp,41900,https://github.com/square/okhttp
7,android,architecture-samples,40500,https://github.com/android/architecture-samples
8,Solido,awesome-flutter,40100,https://github.com/Solido/awesome-flutter
9,square,retrofit,39800,https://github.com/square/retrofit


In [68]:
# creating CSV file of the topic page
get_topic_repos(get_topic_page(topic_urls[4])).to_csv('android.csv', index = None)